線形回帰


使用する特徴ベクトル:
- 曜日
- 当日の売り上げ
- 当日の来客数
- 翌日の観光客予測数
- 食べログの評価
- 1週間の売り上げの平均
- 1週間前の来客数
- イベント情報

目的変数:
- 翌日の来客数

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pylab as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

import math
import sys
%matplotlib inline
sns.set()

In [2]:
sys.path.append('../method/')

In [3]:
import dataset as ds
import preprocessing as pp
import machine_algorithm as ma

In [4]:
data_path = '../csv/wyebiya_data.csv'
df = ds.load_data(data_path)

In [5]:
pd.set_option('display.max_columns', len(df.columns))

In [6]:
df = ds.time_date_sort(df, 'date')

In [7]:
df.head()

,shopId,dayofweek,salesTotal,CustNum,google,kankou,kankou_tomorrow,kankou_after2days,kankou_after3days,kankou_after4days,kankou_after5days,kankou_after6days,tabelog,tabelog_rating,gurunavi,hitosara,weather_today,rain_precipitation_today,rain_precipitation_today_daytime,temp_today_min,temp_today_max,weather_tomorrow,rain_precipitation_tomorrow,rain_precipitation_tomorrow_daytime,temp_tomorrow_min,temp_tomorrow_max,weather_after2days,rainypercent_after2days,temp_after2days_min,temp_after2days_max,weather_after3days,rainypercent_after3days,temp_after3days_min,temp_after3days_max,weather_after4days,rainypercent_after4days,temp_after4days_min,temp_after4days_max,weather_after5days,rainypercent_after5days,temp_after5days_min,temp_after5days_max,weather_after6days,rainypercent_after6days,temp_after6days_min,temp_after6days_max,google_rating,storeId,kankouReal,kankouReal_tomorrow,kankouReal_after2days,kankouReal_after3days,kankouReal_after4days,kankouReal_after5days,kankouReal_after6days,id
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-07-01,1,6,789290.0,368.0,82.0,2339.0,NaN,NaN,NaN,NaN,NaN,NaN,871.0,3.54,221.0,84.0,NaN,NaN,NaN,26.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2017-07-02,1,0,1024830.0,462.0,92.0,1392.0,NaN,NaN,NaN,NaN,NaN,NaN,836.0,3.54,288.0,94.0,NaN,NaN,NaN,28.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2017-07-03,1,1,328250.0,154.0,60.0,773.0,NaN,NaN,NaN,NaN,NaN,NaN,508.0,3.54,217.0,68.0,NaN,NaN,NaN,29.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2017-07-04,1,2,320861.0,158.0,71.0,951.0,NaN,NaN,NaN,NaN,NaN,NaN,626.0,3.54,84.0,111.0,NaN,NaN,NaN,27.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2017-07-05,1,3,359703.0,173.0,86.0,970.0,NaN,NaN,NaN,NaN,NaN,NaN,431.0,3.54,88.0,66.0,NaN,NaN,NaN,26.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [8]:
column_name = "shopId"

df_dining = ds.extract_rows(df, column_name, 1)
df_shop = ds.extract_rows(df, column_name, 2)

In [9]:
df_dining_len = len(df_dining)

In [10]:
# df_dining.head(15)

In [11]:
#  扱う特徴量の選択
# feature = ['dayofweek', 'salesTotal', 'CustNum', 'google', 'kankou', 'tabelog', 'tabelog_rating', 'google_rating']
# feature = ['dayofweek', 'salesTotal', 'CustNum', 'kankou']

# before_feature = ['dayofweek', 'salesTotal', 'CustNum', 'kankou', 'tabelog', 'rain_precipitation_today']
feature = ['dayofweek', 'salesTotal', 'CustNum', 'kankou', 'tabelog']

# before_train_data = df_dining.loc[:,before_feature]
train_data = df_dining.loc[:,feature]

In [12]:
df_precipitation = df_dining.loc[:, 'rain_precipitation_today']

In [13]:
df_precipitation = df_precipitation.fillna(0)

In [14]:
train_data.isnull().sum()

dayofweek      0
salesTotal     5
CustNum        5
kankou         2
tabelog       22
dtype: int64

In [15]:
# 曜日の値が少しずれている部分があるため修正

first_day = train_data['dayofweek'][0]

for date in train_data.index:
    train_data.loc[date, 'dayofweek'] = first_day%7
    first_day += 1

In [16]:
# 前日の値を欠損値に指定
train_data = pp.format_feature(train_data, "tabelog", "before")
# 中央値を欠損値に指定
train_data = pp.format_feature(train_data, "CustNum", "before")
train_data = pp.format_feature(train_data, "salesTotal", "median")
train_data = pp.format_feature(train_data, "kankou", "median")

In [17]:
train_data.isnull().sum()

dayofweek     0
salesTotal    0
CustNum       0
kankou        0
tabelog       0
dtype: int64

### 移動平均を求め，属性ベクトルとして追加する

In [18]:
df_moving_average = train_data.salesTotal

In [19]:
#  売り上げの１週間の移動平均を求める

move_num = 7
df_moving_average = df_moving_average.rolling(window=move_num).mean()

In [20]:
train_data['salesTotal_moving_average'] = df_moving_average

In [21]:
train_data['salesTotal_moving_average']

date
2017-07-01              NaN
2017-07-02              NaN
2017-07-03              NaN
2017-07-04              NaN
2017-07-05              NaN
                  ...      
2020-03-28    596864.571429
2020-03-29    551156.857143
2020-03-30    558101.428571
2020-03-31    518662.285714
2020-04-01    527994.857143
Name: salesTotal_moving_average, Length: 1006, dtype: float64

### 1週間前の来客数を説明変数として用意する

In [22]:
week_ago = train_data['CustNum'].shift(7)

In [23]:
train_data['week_ago'] = week_ago

In [24]:
train_data.head(10)

,dayofweek,salesTotal,CustNum,kankou,tabelog,salesTotal_moving_average,week_ago
date,,,,,,,
2017-07-01,6,789290.0,368.0,2339.0,871.0,NaN,NaN
2017-07-02,0,1024830.0,462.0,1392.0,836.0,NaN,NaN
2017-07-03,1,328250.0,154.0,773.0,508.0,NaN,NaN
2017-07-04,2,320861.0,158.0,951.0,626.0,NaN,NaN
2017-07-05,3,359703.0,173.0,970.0,431.0,NaN,NaN
2017-07-06,4,501675.0,226.0,718.0,673.0,NaN,NaN
2017-07-07,5,401828.0,163.0,1237.0,498.0,532348.142857,NaN
2017-07-08,6,743628.0,334.0,3979.0,932.0,525825.000000,368.0
2017-07-09,0,888732.0,393.0,2146.0,769.0,506382.428571,462.0


### イベント等の影響

In [25]:
df_event_impact = pd.read_csv('../csv/event_impact.csv')

In [26]:
df_event_impact

,date,impact
0,2017-07-01,0.00
1,2017-07-02,0.00
2,2017-07-03,0.00
3,2017-07-04,-3.59
4,2017-07-05,0.00
...,...,...
1001,2020-03-28,-3.59
1002,2020-03-29,-3.59
1003,2020-03-30,0.00
1004,2020-03-31,0.00


In [27]:
# df_event_impact['date'] = df_event_impact['date'].str.replace('/', '-')
# df_event_impact = df_event_impact.replace({'date': {'1': '01','2': '02','3': '03','4': '04','5': '05','6': '06','7': '07','8': '08','9': '09'}})

df_event_impact = df_event_impact.loc[:, ['impact']]
df_event_impact.index = train_data.index

In [28]:
# df_event_correction.to_csv('./correction.csv')

df_event_impact = df_event_impact.loc['2017-07-08':]

In [29]:
# df_event_impact.head(300)
df_event_impact.isnull().sum()

impact    0
dtype: int64

In [30]:
before_train_data = pd.concat([train_data, df_precipitation], axis=1)

In [31]:
# train_data['event_impact'] = df_event_impact

In [32]:
train_data = train_data.dropna(how='any')
before_train_data = before_train_data.dropna(how='any')

In [33]:
# train_data

In [34]:
# before_train_data

### 必要な関数の用意

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso

In [36]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [37]:
# スケーラーの作成
scaler = MinMaxScaler()
event_scaler = MinMaxScaler([-1, 1])

In [38]:
#　訓練，テストデータ数
train_num = 365
test_num = 30

In [39]:
def before_preparation_data(df, scaler):
    X = np.array(df).reshape(-1, len(df.columns))
    X = X[:-1, :]

    y = df['CustNum'][1:]
    y = np.array(y).reshape(-1, 1) 
    
    # スケーリング
    X = pp.feature_scaling(X, scaler)
    y = pp.feature_scaling(y, scaler) 
    
    return X, y, len(X[0])

In [40]:
def preparation_data(df, df_event, scaler, event_scaler):
    
    # 訓練データに結合し，NaNになっている部分を削除する
    df_event = df_event.shift(-1)
    df['event_impact'] = df_event
    df = df.dropna(how='any')
    
    # 影響度とその他の特徴量を分割し，それぞれでスケーリングを行う
    EP = df['event_impact']
    df = df.drop('event_impact', axis=1)
    
    X = np.array(df).reshape(-1, len(df.columns))
    X = X[:-1, :]
    
    y = df['CustNum'][1:]
    y = np.array(y).reshape(-1, 1)
    
    EP = np.array(EP).reshape(-1, 1) 
    EP = EP[:-1, :]
    
    # スケーリング
    # X = pp.feature_scaling(X, scaler)
    # y = pp.feature_scaling(y, scaler)
    
    # EP = pp.feature_scaling(EP, event_scaler)
    
    # X = np.concatenate([X, EP], 1)
    
    return X, y, len(X[0])

In [41]:
def make_dataset(X, y, i, train_num, shift_num):
    start_train = shift_num * i
    end_train = train_num + (shift_num * i)
    start_test = train_num + (shift_num * i)
    end_test = train_num + test_num + (shift_num * i)

    train_X = X[ start_train : end_train,  :]
    test_X = X[ start_test : end_test, :]

    train_y = y[ start_train : end_train,  :]
    test_y = y[ start_test : end_test, :] 
    
    
    return train_X, test_X, train_y, test_y, start_test, end_test

#### 追加

In [42]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import sts

In [47]:
event_kinds = pd.read_csv('../csv/event_jp.csv',encoding="Shift-JIS")
# event_kinds = event_kinds.set_index('date')
# event_kinds = event_kinds['event'].iloc[7:,]
event_kinds = event_kinds.iloc[7:,]

In [48]:
event["event"].unique()

NameError: name 'event' is not defined

In [49]:
event_kinds = event_kinds.drop('date', axis=1)

In [50]:
# train_data2 = train_data.copy()
# before_train_data2 = before_train_data.copy()

train_data2 = train_data.iloc[:,1:5]
before_train_data2 = before_train_data.iloc[:,[1,2,3,4,-1]]
train_data2.head()
# before_train_data2.head()

,salesTotal,CustNum,kankou,tabelog
date,,,,
2017-07-08,743628.0,334.0,3979.0,932.0
2017-07-09,888732.0,393.0,2146.0,769.0
2017-07-10,441044.0,220.0,1418.0,602.0
2017-07-11,413247.0,196.0,1611.0,780.0
2017-07-12,347005.0,152.0,1646.0,841.0


In [51]:
train_data2['event_type'] = event_kinds['event']

In [52]:
train_data2

,salesTotal,CustNum,kankou,tabelog,event_type
date,,,,,
2017-07-08,743628.0,334.0,3979.0,932.0,NaN
2017-07-09,888732.0,393.0,2146.0,769.0,NaN
2017-07-10,441044.0,220.0,1418.0,602.0,NaN
2017-07-11,413247.0,196.0,1611.0,780.0,NaN
2017-07-12,347005.0,152.0,1646.0,841.0,NaN
...,...,...,...,...,...
2020-03-28,560978.0,253.0,6683.0,318.0,NaN
2020-03-29,652850.0,276.0,4156.0,214.0,NaN
2020-03-30,459392.0,206.0,3157.0,230.0,NaN


In [53]:
train_data2["event_type"].unique()

array([nan], dtype=object)

In [54]:
# day1_cast = train_data2['CustNum'].shift()
day1_sales = train_data2['salesTotal'].shift()
day1_kankou = train_data2['kankou'].shift(-1) #翌日の観光客予測数のため
day1_tabelog = train_data2['tabelog'].shift()


# train_data2['day1_cast'] = day1_cast
train_data2['day1_sales'] = day1_sales
train_data2['day1_kankou'] = day1_kankou
train_data2['day1_tabelog'] = day1_tabelog

In [55]:
pd.set_option('display.max_rows', 1000)
# train_data2.head(600)

### 予測

In [56]:
# 訓練データに結合し，NaNになっている部分を削除する
# df_event = df_event_impact.shift(-1)

# train_data2['event_impact'] = df_event
train_data2['event_impact'] = df_event_impact

#train_data2 = train_data2.dropna(how='any')

# 影響度とその他の特徴量を分割し，それぞれでスケーリングを行う
EP = train_data2['event_impact']
train_data5 = train_data2.drop('event_impact', axis=1)

X = np.array(df).reshape(-1, len(df.columns))
X = X[:-1, :]

y = train_data2['CustNum'][1:]
y = np.array(y).reshape(-1, 1)

EP = np.array(EP).reshape(-1, 1)
EP = EP[:-1, :]


In [57]:
# train_data2

In [58]:
train_y_list = []
train_x_list = []
test_y_list = []
x_first = 0
y_first = 366
for i in range(20):
    train_X = train_data2.iloc[:,[0,2,3]][x_first:y_first-1]
    train_y = train_data2["CustNum"][x_first:y_first-1]
    test_y = train_data2["CustNum"][y_first:y_first+30]
    train_x_list.append(train_X)
    train_y_list.append(train_y)
    test_y_list.append(test_y)
    x_first += 30
    y_first += 30

In [59]:

train_x_list[1].tail()

,salesTotal,kankou,tabelog
date,,,
2018-08-02,547280.0,6259.0,657.0
2018-08-03,665290.0,6749.0,676.0
2018-08-04,860890.0,7727.0,656.0
2018-08-05,969020.0,7210.0,693.0
2018-08-06,665270.0,6890.0,930.0


In [60]:
# train3 = []
# test3 = []
# x_first = 0
# y_first = 366
# for i in range(20):
#     train_X = train_data2[x_first:y_first-1]
#     # test_y = train_data2["CustNum"][y_first:y_first+30]
#     test_y = train_data2[y_first:y_first+30]
#     train3.append(train_X)
#     test3.append(test_y)
#     x_first += 30
#     y_first += 30

In [61]:
train_data2.to_csv('../data.csv')


In [62]:
len(train_data2)

999

In [63]:
# データ分割 train: 365 , test: 30
train3 = []
test3 = []
x_first = 0
y_first = 365
for i in range(22):
    train_X = train_data2[x_first:y_first]
    # test_y = train_data2["CustNum"][y_first:y_first+30]
    test_y = train_data2[y_first:y_first+30]
    train3.append(train_X)
    test3.append(test_y)
    x_first += 30
    y_first += 30

In [64]:
# データ分割 train: 730 , test: 60
train4 = []
test4 = []
x_first = 0
y_first = 730
for i in range(5):
    train_X = train_data2[x_first:y_first]
    # test_y = train_data2["CustNum"][y_first:y_first+30]
    test_y = train_data2[y_first:y_first+60]
    train4.append(train_X)
    test4.append(test_y)
    x_first += 60
    y_first += 60

In [57]:
# データ分割 train: 182 , test: 15
train_182_15 = []
test_182_15 = []
x_first = 0
y_first = 182
for i in range(55):
    train_X = train_data2[x_first:y_first]
    # test_y = train_data2["CustNum"][y_first:y_first+30]
    test_y = train_data2[y_first:y_first+60]
    train_182_15.append(train_X)
    test_182_15.append(test_y)
    x_first += 15
    y_first += 15

In [58]:
train3[1].tail()

,salesTotal,CustNum,kankou,tabelog,day1_sales,day1_kankou,day1_tabelog,event_impact
date,,,,,,,,
2018-08-02,547280.0,246.0,6259.0,657.0,566980.0,6749.0,621.0,0.0
2018-08-03,665290.0,305.0,6749.0,676.0,547280.0,7727.0,657.0,0.0
2018-08-04,860890.0,393.0,7727.0,656.0,665290.0,7210.0,676.0,0.0
2018-08-05,969020.0,452.0,7210.0,693.0,860890.0,6890.0,656.0,0.0
2018-08-06,665270.0,284.0,6890.0,930.0,969020.0,7634.0,693.0,0.0


In [59]:
test3[1].head()

,salesTotal,CustNum,kankou,tabelog,day1_sales,day1_kankou,day1_tabelog,event_impact
date,,,,,,,,
2018-08-07,561530.0,238.0,7634.0,745.0,665270.0,7919.0,930.0,-3.59
2018-08-08,795010.0,1137.0,7919.0,696.0,561530.0,9906.0,745.0,0.00
2018-08-09,624250.0,314.0,9906.0,752.0,795010.0,9631.0,696.0,0.00
2018-08-10,766360.0,378.0,9631.0,942.0,624250.0,12501.0,752.0,0.00
2018-08-11,1153980.0,445.0,12501.0,1141.0,766360.0,11722.0,942.0,0.00


In [58]:
# def bunkatsu(train):
#     train3 = []
#     test3 = []
#     x_first = 0
#     y_first = 366
#     for i in range(20):
#         train_X = train[x_first:y_first-1]
#         # test_y = train_data2["CustNum"][y_first:y_first+30]
#         test_y = train[y_first:y_first+30]
#         train3.append(train_X)
#         test3.append(test_y)
#         x_first += 30
#         y_first += 30


In [59]:
# train3[7].to_csv('../sales_Cust_kankou_event/train.csv')
# test3[7].to_csv('../sales_Cust_kankou_event/test.csv')

In [60]:
train3[0].to_csv('../train/train1.csv')
train3[1].to_csv('../train/train2.csv')
train3[2].to_csv('../train/train3.csv')
train3[3].to_csv('../train/train4.csv')
train3[4].to_csv('../train/train5.csv')
train3[5].to_csv('../train/train6.csv')
train3[6].to_csv('../train/train7.csv')
train3[7].to_csv('../train/train8.csv')
train3[8].to_csv('../train/train9.csv')
train3[9].to_csv('../train/train10.csv')
train3[10].to_csv('../train/train11.csv')
train3[11].to_csv('../train/train12.csv')
train3[12].to_csv('../train/train13.csv')
train3[13].to_csv('../train/train14.csv')
train3[14].to_csv('../train/train15.csv')
train3[15].to_csv('../train/train16.csv')
train3[16].to_csv('../train/train17.csv')
train3[17].to_csv('../train/train18.csv')
train3[18].to_csv('../train/train19.csv')
train3[19].to_csv('../train/train20.csv')
train3[20].to_csv('../train/train21.csv')
train3[21].to_csv('../train/train22.csv')




In [61]:
test3[0].to_csv('../test/test1.csv')
test3[1].to_csv('../test/test2.csv')
test3[2].to_csv('../test/test3.csv')
test3[3].to_csv('../test/test4.csv')
test3[4].to_csv('../test/test5.csv')
test3[5].to_csv('../test/test6.csv')
test3[6].to_csv('../test/test7.csv')
test3[7].to_csv('../test/test8.csv')
test3[8].to_csv('../test/test9.csv')
test3[9].to_csv('../test/test10.csv')
test3[10].to_csv('../test/test11.csv')
test3[11].to_csv('../test/test12.csv')
test3[12].to_csv('../test/test13.csv')
test3[13].to_csv('../test/test14.csv')
test3[14].to_csv('../test/test15.csv')
test3[15].to_csv('../test/test16.csv')
test3[16].to_csv('../test/test17.csv')
test3[17].to_csv('../test/test18.csv')
test3[18].to_csv('../test/test19.csv')
test3[19].to_csv('../test/test20.csv')
test3[20].to_csv('../test/test21.csv')
test3[21].to_csv('../test/test22.csv')


In [62]:
train4[0].to_csv('../train/2train1.csv')
train4[1].to_csv('../train/2train2.csv')
train4[2].to_csv('../train/2train3.csv')
train4[3].to_csv('../train/2train4.csv')
train4[4].to_csv('../train/2train5.csv')


    
test4[0].to_csv('../test/2test1.csv')
test4[1].to_csv('../test/2test2.csv')
test4[2].to_csv('../test/2test3.csv')
test4[3].to_csv('../test/2test4.csv')
test4[4].to_csv('../test/2test5.csv')


In [101]:
import os

for i, test in enumerate(test_182_15):
    if not os.path.exists('../train/182_15/'):
        os.makedirs('../train/182_15/')
    if not os.path.exists('../test/182_15/'):
        os.makedirs('../test/182_15/')
    train_182_15[i].to_csv('../train/182_15/train' + str(i+1) + '.csv')
    test.to_csv('../test/182_15/test' + str(i+1) + '.csv')
